In [15]:
import time
from datetime import date
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [16]:
today = date.today()
print(str(today), type(today))
today = str(today).replace("-", "_")
print(today, type(today))

2024-07-29 <class 'datetime.date'>
2024_07_29 <class 'str'>


In [17]:
# 내용 안나옴
# url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
# payload = dict(method='loadInitPage')
# r = requests.get(url, params=payload)
# print(r.status_code)
# print(r.text)

In [18]:
page = 1
page_size = 100
final_result_df = pd.DataFrame()
while True:
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method='searchCorpList',pageIndex=page, currentPageSize=page_size,orderMode=3,orderStat='D',searchType=13, fiscalYearEnd='all', location='all')
    r = requests.get(url, params=payload)
    print(r.status_code, end="\r")
    soup = bs(r.text, 'lxml')
    total_items = int(soup.select_one(".info.type-00 > em").text.replace(",", ""))
    total_pages = total_items // page_size + 1
    print(f"{page}/{total_pages} 수집중", end="\r")
    keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")  
    result = {}
    for tr in soup.select('tr'):
        for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
            if idx == 0:
                kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
                kind = ", ".join(kinds)
                code = td.select_one("a")['onclick'].split("'")[1] # 종목코드 추출
                result.setdefault('증권종류', []).append(kind)
                result.setdefault(key, []).append(td.text)
                result.setdefault('종목코드', []).append(code+"0") # 종목코드 추가
            elif idx == 6:
                home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
                result.setdefault(key, []).append(home_link)
            else:
                result.setdefault(key, []).append(td.text)
    result_df = pd.DataFrame(result)
    final_result_df = pd.concat([final_result_df, result_df])
        
    if page < total_pages:
        page += 1
        time.sleep(5)
    else:
        break

display(final_result_df)
final_result_df.to_csv(f"상장기업정보_{today}기준.csv", encoding='utf-8', index=False)  # 수집일 기준으로 csv 파일로 저장



,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,산일전기,062040,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","유입, 몰드, 주상, 건식 변압기 등",2024-07-29,12월,박동석,http://www.sanil.co.kr,경기도
1,유가증권,에이치에스효성,487570,기타 금융업,지주사업,2024-07-29,12월,조현상..,http://www.hshyosung.com,서울특별시
2,"코스닥, 투자주의종목",엔에이치스팩31호,481890,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,,서울특별시
3,코스닥,SK증권제13호스팩,473950,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,,서울특별시
4,코스닥,엑셀세라퓨틱스,373110,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,,서울특별시
...,...,...,...,...,...,...,...,...,...,...
0,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
1,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
3,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


# DB에 상장기업정보 저장하기

In [22]:
import os
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pymysql
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path=".env_db")


True

In [20]:
# SQLAlchemy 엔진 생성
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('user')}:{os.getenv('pw')}@{os.getenv('host')}")
conn = engine.connect()

# 데이터베이스에 지정한 데이터베이스 이름 생성
conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {os.getenv('database')}"))
conn.close()

# DataFrame을 MySQL 테이블로 저장
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('user')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()
final_result_df.to_sql(f'{today[:7]}_stock_company_info', con=conn, if_exists='append', index=False)
conn.close()

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:None.None

In [23]:
today = today.replace("-", "_")
today

'2024_07_29'

In [24]:
# DataFrame을 MySQL 테이블로 저장

engine = create_engine(f'{os.getenv('db')} + {os.getenv('dbtype')}://root:1234@127.0.0.1:3306/korea_stock_info')
conn = engine.connect()
final_result_df.to_sql(f'{today}_stock_company_info', con=conn, if_exists = 'append', index = False)
conn.close()

In [26]:
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()
final_result_df.to_sql(f'{today}_stock_company_info2', con=conn, if_exists = 'append', index = False)
conn.close()

In [ ]:
pd.read_csv("./상장기업정보2024-07-29_기준")

In [ ]:
keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")

In [ ]:
result = {}
for tr in soup.select('tr'):
    for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
        if idx == 0:
            kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
            kind = ", ".join(kinds)
            code = td.select_one("a")['onclick'].split("'")[1] # 종목코드 추출
            result.setdefault('증권종류', []).append(kind)
            result.setdefault(key, []).append(td.text)
            result.setdefault('종목코드', []).append(code)
        elif idx == 6:
            home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
            result.setdefault(key, []).append(home_link)
        else:
            result.setdefault(key, []).append(td.text)

        
result
result_df = pd.DataFrame(result)
result_df            
       



# pandas read_html로 표 데이터 가져오기

In [3]:
page = 1
page_size = 100
final_result_df = pd.DataFrame()

url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method='searchCorpList',pageIndex=page, currentPageSize=page_size,orderMode=3,orderStat='D',searchType=13, fiscalYearEnd='all', location='all')
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.text, 'lxml')

https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=1&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200


In [20]:
# 종목코드
for tr in soup.select('tr')[:1]:
    for td in soup.select('tr')[1:2]:
        print(td.select_one("a")['onclick'].split("'")[1])

48189


In [80]:
df = pd.read_html(r.text)
df[0]

C:\Users\5-29\AppData\Local\Temp\ipykernel_6828\490771583.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(r.text)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,엔에이치스팩31호,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,NaN,서울특별시
1,SK증권제13호스팩,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,NaN,서울특별시
2,엑셀세라퓨틱스,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,NaN,서울특별시
3,이베스트스팩6호,금융 지원 서비스업,기업인수목적 주식회사,2024-07-12,12월,박용,NaN,서울특별시
4,시프트업,소프트웨어 개발 및 공급업,게임 소프트웨어,2024-07-11,12월,김형태,홈페이지 보기,서울특별시
...,...,...,...,...,...,...,...,...
95,에코프로머티,일차전지 및 이차전지 제조업,배터리 양극재용 핵심소재 하이니켈 전구체,2023-11-17,12월,김병훈,홈페이지 보기,경상북도
96,캡스톤파트너스,기타 금융업,벤처투자조합 등 조합 결성 및 운영,2023-11-15,12월,송은강,홈페이지 보기,서울특별시
97,프로젠,자연과학 및 공학 연구개발업,바이오의약품,2023-11-15,12월,김종균,홈페이지 보기,서울특별시
98,에스와이스틸텍,"구조용 금속제품, 탱크 및 증기발생기 제조업",데크플레이트,2023-11-13,12월,홍성균,홈페이지 보기,충청북도
